# download stock price

In [12]:
import os
import json
import time
import requests
import base64
import pandas as pd

In [13]:
def call_refresh_api(refreshtoken: str):
    """
    idTokenをリフレッシュするメソッド。

    Parameters
    ----------
    refreshtoken : str
        refreshtoken。ログイン後の画面からご確認いただけます。

    Returns
    -------
    resjson : dict
        新しいidtokenが格納されたAPIレスポンス(json形式)
    """
    headers = {"accept": "application/json"}
    data = {"refresh-token": refreshtoken}

    response = requests.post(
        "https://api.jpx-jquants.com/refresh", headers=headers, data=json.dumps(data)
    )

    resjson = json.loads(response.text)
    return resjson

In [15]:
def call_jquants_api(params: dict, idtoken: str, apitype: str, code: str = None):
    """
    J-QuantsのAPIを試すメソッド。

    Parameters
    ----------
    params : dict
        リクエストパラメータ。
    idtoken : str
        idTokenはログイン後の画面からご確認いただけます。
    apitype: str
        APIの種類。"news", "prices", "lists"などがあります。
    code: str
        銘柄を指定するAPIの場合に設定します。

    Returns
    -------
    resjson : dict
        APIレスポンス(json形式)
    """
    datefrom = params.get("datefrom", None)
    dateto = params.get("dateto", None)
    date = params.get("date", None)
    includedetails = params.get("includedetails", "false")
    keyword = params.get("keyword", None)
    headline = params.get("headline", None)
    paramcode = params.get("code", None)
    nexttoken = params.get("nextToken", None)
    headers = {"accept": "application/json", "Authorization": idtoken}
    data = {
        "from": datefrom,
        "to": dateto,
        "includeDetails": includedetails,
        "nextToken": nexttoken,
        "date": date,
        "keyword": keyword,
        "headline": headline,
        "code": paramcode,
    }

    if code:
        code = "/" + code
        r = requests.get(
            "https://api.jpx-jquants.com/" + apitype + code,
            params=data,
            headers=headers,
        )
    else:
        r = requests.get(
            "https://api.jpx-jquants.com/" + apitype, params=data, headers=headers
        )
    resjson = json.loads(r.text)
    return resjson

In [25]:
month = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
year = ['2016', '2017', '2018', '2019', '2020',  '2021']
day = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']

results = []
for y in year:
    # 1h有効なtoken get
    refreshtoken = 'eyJjdHkiOiJKV1QiLCJlbmMiOiJBMjU2R0NNIiwiYWxnIjoiUlNBLU9BRVAifQ.V-vtlyD4PIK_-iF_AJyr2r5fPYS13hQ7KLuiHv90wQbrpcPpcoFMlKuvrC0_wxqxIa8bpdKjBTh8E45Zuwa3IjwavI6h7l6fmDRmpy6eRFI-B0rtNFot24aeQfaHqmtgh58tARVkbCEfbMZruXvsftRjBXwleUE75g28P-qQJ6cJtR0DfCGblJNKxmjWPrGuvupGadILS0unFK5IJf_xe0KPsUeJEezlkm4PO3Tccyy9MOFGb-2PWvvsjvl3WAcYT7XTBqVQ3v4-EhnBXhmHzsa95D1GcI9v5dsV1e5whHnzMdfHnFVNY9wCIYPGftItZrwabkK6YrZymVED9ISMWA.mZp0m_oU9l-491jf.sBq1joc1a_Wi0K7uz0nTzzxt2nupF_UYhw_FOXvylukhLoby91b2rKjBvmuwSOXNKy3DfCVbKhaMzdVRxDl6mbmvsIPIJSY8FMHGnE7entw_JCbwBJdaW1tP_sPIgGbcJeL4urNaeO5sczTM36Yn6E9G_C8larNon0aFxHjLUZpIB81e3DY4vNZevQc7B1kfaCboSRoL-WikFQAmDTp_VebuXFDK6MWTOB_Ek5tiBgXCfylyGAUOlFDOHjd82T_GvvCf_Zzi-ECPkU6tTnQCwqId3Vn7Knb4WzY6JWb4fx3qX3j1y7XA-r8PDs-a9_PyFvz-wApMxuc--9pAH60HLKBEdApoQW9lGo4aUSHOrGWMpidI9Hgc_rJsNuoBJ2Eib0oWvQSe5XHePbAxsHdCfsVOIT1XhkxO_U6FvpWhZGsLDTgo5G5TpwugGZj8qTzOmIHG1lpsnEKI471TxxD_LQ67wKbOd8lNUqKGxKS1jdqOIrnbjqkKgJScH8lURC8efy1FXZdWVDZOQxskukM3XkJybc9MrqSZrxgm1QMpLpWF3GybHrPsl5ukU1I5jlupF2UtCS_rdRdVpqHhVN-7WXgklNruQgzLqnHjri5ApeXqHCjnNYrQi2wsiM2oZr2tNqw5GESlIrq_5P6SYZUwhDpRzmZBIuC65t4OpKdczyUayqkOPaRRmAMkFWsv0itH5CsZYZlFPloCwxHS2Y9V2m6dSMvjIKqmTtw4f5pLRDN5WS2GuWBLa5_em4cjH_co_ygTGfknPObGPRN1SI1dVNrJNEWv7F4_8l01fbRX9dthm-IF394OR8Mur8dLhFrRrbH-uylbO6v37PU-iD8Z4RPstRLKTM9KOpWpdebdUAwue1-sIT4Xta3epsvGWchCfie00H7MvX0DonGS82Tke8k_cXunjXyOGsopYQgC6P5IE-cy0LJojrFFxeTPyegtnrKIenKiCLI2fQ9w80TAaZaLk71BgkJjBNyzd9gqKwFy_3Nn0ST03tzF3gxIdMSK-z3AFEiRaMMvWrFE2k1ArjeJ2oSpNhabjwLA57hXfRjqHHEGbfj0-qlkBkDNodgxfj8jl5OK96fAuni1goGslhEFIG58thmStLXg9QvZ7G5pno8e0f0imzhoHQxPHA-YfZxPhJQxNsgsX1vujJalCLp5elguUbLxItHZuBCmxwrPoD5xHDHyA3__wM0uDJ8guh6b4vp1WxMxEait89SoDZdjQ0s5IJBrXqxHozu50v0wwtPlJescLxTkXoKeuk4tiETmJCfz2_ITHW5zYS83mlwO0vUJPNIoXzblcY4vPiOXufNI8Omb8Hzha9mHwpwMZwwp3gqHD5o7.ifY8YH-HcgDnNd3UZV-Saw'
    ret = call_refresh_api(refreshtoken)
    id_token = ret['idToken']
    for m in month:
        for d in day:
            next_flg = True
            next_token = None
            while next_flg:
                date = f"{y}-{m}-{d}"
                print(date)
                if (y == '2021') & (m=='05'):
                    print(f'breake {date}')
                    break
                paramdict = {}
                paramdict["date"] = date
                paramdict["includedetails"] = "True"
                paramdict["nextToken"] = next_token
                ret = call_jquants_api(paramdict, id_token, "prices")
                results.append(pd.DataFrame(ret['prices']))
                if 'nextToken' in ret.keys():
                    next_token = ret['nextToken']
                else:
                    next_flg = False
                time.sleep(0.5)

2016-01-01
2016-01-02
2016-01-03
2016-01-04
2016-01-04
2016-01-05
2016-01-05
2016-01-06
2016-01-06
2016-01-07
2016-01-07
2016-01-08
2016-01-08
2016-01-09
2016-01-10
2016-01-11
2016-01-12
2016-01-12
2016-01-13
2016-01-13
2016-01-14
2016-01-14
2016-01-15
2016-01-15
2016-01-16
2016-01-17
2016-01-18
2016-01-18
2016-01-19
2016-01-19
2016-01-20
2016-01-20
2016-01-21
2016-01-21
2016-01-22
2016-01-22
2016-01-23
2016-01-24
2016-01-25
2016-01-25
2016-01-26
2016-01-26
2016-01-27
2016-01-27
2016-01-28
2016-01-28
2016-01-29
2016-01-29
2016-01-30
2016-01-31
2016-02-01
2016-02-01
2016-02-02
2016-02-02
2016-02-03
2016-02-03
2016-02-04
2016-02-04
2016-02-05
2016-02-05
2016-02-06
2016-02-07
2016-02-08
2016-02-08
2016-02-09
2016-02-09
2016-02-10
2016-02-10
2016-02-11
2016-02-12
2016-02-12
2016-02-13
2016-02-14
2016-02-15
2016-02-15
2016-02-16
2016-02-16
2016-02-17
2016-02-17
2016-02-18
2016-02-18
2016-02-19
2016-02-19
2016-02-20
2016-02-21
2016-02-22
2016-02-22
2016-02-23
2016-02-23
2016-02-24
2016-02-24

2017-03-16
2017-03-17
2017-03-17
2017-03-18
2017-03-19
2017-03-20
2017-03-21
2017-03-21
2017-03-22
2017-03-22
2017-03-23
2017-03-23
2017-03-24
2017-03-24
2017-03-25
2017-03-26
2017-03-27
2017-03-27
2017-03-28
2017-03-28
2017-03-29
2017-03-29
2017-03-30
2017-03-30
2017-03-31
2017-03-31
2017-04-01
2017-04-02
2017-04-03
2017-04-03
2017-04-04
2017-04-04
2017-04-05
2017-04-05
2017-04-06
2017-04-06
2017-04-07
2017-04-07
2017-04-08
2017-04-09
2017-04-10
2017-04-10
2017-04-11
2017-04-11
2017-04-12
2017-04-12
2017-04-13
2017-04-13
2017-04-14
2017-04-14
2017-04-15
2017-04-16
2017-04-17
2017-04-17
2017-04-18
2017-04-18
2017-04-19
2017-04-19
2017-04-20
2017-04-20
2017-04-21
2017-04-21
2017-04-22
2017-04-23
2017-04-24
2017-04-24
2017-04-25
2017-04-25
2017-04-26
2017-04-26
2017-04-27
2017-04-27
2017-04-28
2017-04-28
2017-04-29
2017-04-30
2017-04-31
2017-05-01
2017-05-01
2017-05-02
2017-05-02
2017-05-03
2017-05-04
2017-05-05
2017-05-06
2017-05-07
2017-05-08
2017-05-08
2017-05-09
2017-05-09
2017-05-10

2018-05-31
2018-06-01
2018-06-01
2018-06-02
2018-06-03
2018-06-04
2018-06-04
2018-06-05
2018-06-05
2018-06-06
2018-06-06
2018-06-07
2018-06-07
2018-06-08
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-11
2018-06-12
2018-06-12
2018-06-13
2018-06-13
2018-06-14
2018-06-14
2018-06-15
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-18
2018-06-19
2018-06-19
2018-06-20
2018-06-20
2018-06-21
2018-06-21
2018-06-22
2018-06-22
2018-06-23
2018-06-24
2018-06-25
2018-06-25
2018-06-26
2018-06-26
2018-06-27
2018-06-27
2018-06-28
2018-06-28
2018-06-29
2018-06-29
2018-06-30
2018-06-31
2018-07-01
2018-07-02
2018-07-02
2018-07-03
2018-07-03
2018-07-04
2018-07-04
2018-07-05
2018-07-05
2018-07-06
2018-07-06
2018-07-07
2018-07-08
2018-07-09
2018-07-09
2018-07-10
2018-07-10
2018-07-11
2018-07-11
2018-07-12
2018-07-12
2018-07-13
2018-07-13
2018-07-14
2018-07-15
2018-07-16
2018-07-17
2018-07-17
2018-07-18
2018-07-18
2018-07-19
2018-07-19
2018-07-20
2018-07-20
2018-07-21
2018-07-22
2018-07-23
2018-07-23

2019-08-16
2019-08-17
2019-08-18
2019-08-19
2019-08-19
2019-08-20
2019-08-20
2019-08-21
2019-08-21
2019-08-22
2019-08-22
2019-08-23
2019-08-23
2019-08-24
2019-08-25
2019-08-26
2019-08-26
2019-08-27
2019-08-27
2019-08-28
2019-08-28
2019-08-29
2019-08-29
2019-08-30
2019-08-30
2019-08-31
2019-09-01
2019-09-02
2019-09-02
2019-09-03
2019-09-03
2019-09-04
2019-09-04
2019-09-05
2019-09-05
2019-09-06
2019-09-06
2019-09-07
2019-09-08
2019-09-09
2019-09-09
2019-09-10
2019-09-10
2019-09-11
2019-09-11
2019-09-12
2019-09-12
2019-09-13
2019-09-13
2019-09-14
2019-09-15
2019-09-16
2019-09-17
2019-09-17
2019-09-18
2019-09-18
2019-09-19
2019-09-19
2019-09-20
2019-09-20
2019-09-21
2019-09-22
2019-09-23
2019-09-24
2019-09-24
2019-09-25
2019-09-25
2019-09-26
2019-09-26
2019-09-27
2019-09-27
2019-09-28
2019-09-29
2019-09-30
2019-09-30
2019-09-31
2019-10-01
2019-10-01
2019-10-02
2019-10-02
2019-10-03
2019-10-03
2019-10-04
2019-10-04
2019-10-05
2019-10-06
2019-10-07
2019-10-07
2019-10-08
2019-10-08
2019-10-09

2020-11-02
2020-11-03
2020-11-04
2020-11-04
2020-11-05
2020-11-05
2020-11-06
2020-11-06
2020-11-07
2020-11-08
2020-11-09
2020-11-09
2020-11-10
2020-11-10
2020-11-11
2020-11-11
2020-11-12
2020-11-12
2020-11-13
2020-11-13
2020-11-14
2020-11-15
2020-11-16
2020-11-16
2020-11-17
2020-11-17
2020-11-18
2020-11-18
2020-11-19
2020-11-19
2020-11-20
2020-11-20
2020-11-21
2020-11-22
2020-11-23
2020-11-24
2020-11-24
2020-11-25
2020-11-25
2020-11-26
2020-11-26
2020-11-27
2020-11-27
2020-11-28
2020-11-29
2020-11-30
2020-11-30
2020-11-31
2020-12-01
2020-12-01
2020-12-02
2020-12-02
2020-12-03
2020-12-03
2020-12-04
2020-12-04
2020-12-05
2020-12-06
2020-12-07
2020-12-07
2020-12-08
2020-12-08
2020-12-09
2020-12-09
2020-12-10
2020-12-10
2020-12-11
2020-12-11
2020-12-12
2020-12-13
2020-12-14
2020-12-14
2020-12-15
2020-12-15
2020-12-16
2020-12-16
2020-12-17
2020-12-17
2020-12-18
2020-12-18
2020-12-19
2020-12-20
2020-12-21
2020-12-21
2020-12-22
2020-12-22
2020-12-23
2020-12-23
2020-12-24
2020-12-24
2020-12-25

In [26]:
df = pd.concat(results, axis=0)

In [27]:
df.to_csv('data/stock_price/stock_price.csv.gz', index=False, compression='gzip')

In [28]:
new_df = pd.read_csv('data/stock_price/stock_price.csv.gz')
new_df.tail()

,EndOfDayQuote Open,EndOfDayQuote PreviousClose,EndOfDayQuote CumulativeAdjustmentFactor,EndOfDayQuote VWAP,EndOfDayQuote Low,EndOfDayQuote PreviousExchangeOfficialClose,EndOfDayQuote High,EndOfDayQuote Date,EndOfDayQuote Close,EndOfDayQuote PreviousExchangeOfficialCloseDate,EndOfDayQuote ExchangeOfficialClose,EndOfDayQuote ChangeFromPreviousClose,EndOfDayQuote PercentChangeFromPreviousClose,EndOfDayQuote PreviousCloseDate,Local Code,EndOfDayQuote Volume
4525886,1722.0,1719.0,1.0,1711.462,1707.0,1719.0,1724.0,2021/04/30,1707.0,2021/04/28,1707.0,-12.0,-0.698,2021/04/28,9993,5200.0
4525887,2336.0,2336.0,1.0,2325.642,2315.0,2336.0,2337.0,2021/04/30,2315.0,2021/04/28,2315.0,-21.0,-0.899,2021/04/28,9994,5300.0
4525888,428.0,430.0,1.0,424.865,421.0,430.0,430.0,2021/04/30,421.0,2021/04/28,421.0,-9.0,-2.093,2021/04/28,9995,83100.0
4525889,1501.0,1502.0,1.0,1496.667,1494.0,1502.0,1501.0,2021/04/30,1494.0,2021/04/28,1494.0,-8.0,-0.533,2021/04/28,9996,300.0
4525890,1165.0,1168.0,1.0,1163.398,1157.0,1168.0,1173.0,2021/04/30,1159.0,2021/04/28,1159.0,-9.0,-0.771,2021/04/28,9997,251300.0
